# Exploratory Text Analysis

## What kinds of text analysis are there?

* analyst knows the pattern
    * regular expressions
* analyst does not know the pattern
    * natural language processing
        * compares historical examples to judge novel cases
            * comparisons are statistical and approximate
            

### Examples of Analysis

When you know the pattern:

In [11]:
pattern = '£ ?[0-9][0-9]?' # £ then SPACE-optional then digit then digit-optional 

document = 'My eggs cost £3, bread cost £2, vodka cost £35'

In [12]:
import re

In [13]:
re.findall(pattern, document)

['£3', '£2', '£35']

If you dont:

* sentiment analysis
    * how positive/negative is this (new) review?
* topic analysis 
    * what is this document about?

## What can I do if I know what pattern I want to find?

* finding ("extracting")
    * what matches the pattern?
* matching ("validating")
    * does the entire document match YES/NO?
* substitue ("replacing")
    * replace a part that matches a pattern with another...

## How do I validate text with pandas?

In [16]:
import pandas as pd

In [19]:
ti = pd.read_csv('../datasets/titanic.csv')
ti.sample(1)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
711,0,1,male,NaN,0,0,26.55,S,First,man,True,C,Southampton,no,True


In [133]:
ti['ticket'] = "Ticket: " + ti['class'] + "; Price: $ " + ti['fare'].astype(str) + "; Port: " + ti['embark_town'] + ";"

In [134]:
ti[['class', 'fare', 'embark_town', 'ticket']].head(3)

,class,fare,embark_town,ticket
0,Third,7.2500,Southampton,Ticket: Third; Price: $ 7.25; Port: Southampton;
1,First,71.2833,Cherbourg,Ticket: First; Price: $ 71.2833; Port: Cherbourg;
2,Third,7.9250,Southampton,Ticket: Third; Price: $ 7.925; Port: Southampton;


In [135]:
pattern = '(First|Second)'

ti['class'].str.match(pattern)

0      False
1       True
2      False
3       True
4      False
       ...  
886     True
887     True
888    False
889     True
890    False
Name: class, Length: 891, dtype: bool

In [136]:
ti.loc[ ti['class'].str.match(pattern)  , 'survived'].mean()

0.5575

## How do I extract data with pandas?

In [137]:
ti[['class', 'fare', 'embark_town', 'ticket']].head(3)

,class,fare,embark_town,ticket
0,Third,7.2500,Southampton,Ticket: Third; Price: $ 7.25; Port: Southampton;
1,First,71.2833,Cherbourg,Ticket: First; Price: $ 71.2833; Port: Cherbourg;
2,Third,7.9250,Southampton,Ticket: Third; Price: $ 7.925; Port: Southampton;


In [138]:
pattern = '([0-9.]+)'

ti['ticket'].str.extract(pattern).sample(4)

,0
702,14.4542
799,24.15
414,7.925
575,14.5


## How do I substitue text with pandas?

In [139]:
ti['ticket'].str.replace('$', '€').sample(1)

890    Ticket: Third; Price: € 7.75; Port: Queenstown;
Name: ticket, dtype: object

## What are regular expressions?

Regular expressions are a language for describing patterns in text. 

They are separate from python, but may be used within python program. (And elsewhere, eg., often in SQL). 

They are notoriously difficult to read and write; and as a separate language, an additional tool to learn. 

## What regular expression patterns can I use?

* literals
    * `a`, find me an `a`
    * `£`, find `£`
    * `!` means `!` 
    * ... most symbols mean "find me"
* `.`
    * find any **single** symbol 
* character classes -- find a **single** symbol
    * `[abc]` $\rightarrow$ **either** a, b, c
    * `[0-9]` $\rightarrow$ **either** 0, 1, 2, 3,...9
    * `[A-Z]` $\rightarrow$ **either** capital A, B, ... Z
    * inversions
        * `[^abc]` $\rightarrow$ **is not** `a` OR `b` OR `c`
        * `[^a-zA-Z0-9 ]`  $\rightarrow$ **is not** alphanumeric-ish
    
* alternatives -- find the character**s** given by...
    * `(May|June|July)`  $\rightarrow$ **the whole worlds** May OR June..
    

In [140]:
ti['ticket'].str.extract('(Ticket: (First|Second))')

,0,1
0,NaN,NaN
1,Ticket: First,First
2,NaN,NaN
3,Ticket: First,First
4,NaN,NaN
...,...,...
886,Ticket: Second,Second
887,Ticket: First,First
888,NaN,NaN
889,Ticket: First,First


In [141]:
ti['ticket'].str.extract('( [0-9][0-9])')

,0
0,NaN
1,71
2,NaN
3,53
4,NaN
...,...
886,13
887,30
888,23
889,30


In [142]:
ti['ticket'].sample(1)

352    Ticket: Third; Price: $ 7.2292; Port: Cherbourg;
Name: ticket, dtype: object

In [143]:
ti['ticket'].str.extract('(Ticket: [A-Z])').sample(2)

,0
48,Ticket: T
347,Ticket: T


In [144]:
ti['ticket'].str.extract('(T........)').sample(3)

,0
265,Ticket: S
206,Ticket: T
602,Ticket: F


In [145]:
ti['ticket'].str.extract('(Price: [^0-9A-Za-z] ..)').sample(3)

,0
776,Price: $ 7.
832,Price: $ 7.
139,Price: $ 79


In [146]:
ti['ticket'].str.extract('(Port: (Cherbourg|Southampton))').sample(3)

,0,1
458,Port: Southampton,Southampton
612,NaN,NaN
595,Port: Southampton,Southampton


* repetitions
    * optional `?`
        * an optional number: `[0-9]?`
    * one or more `+`
        * one or more spaces: ` +`  
    * optional, or more, `*`
        * ` [0-9][0-9]?.[0-9]*`
    

In [147]:
ti['ticket'].str.extract('([0-9][0-9]?.[0-9]*)').sample(3)

,0
683,46.9
598,7.225
93,20.575


In [148]:
ti['ticket'].str.extract('(Ticket: [a-zA-Z]+)').sample(3)

,0
677,Ticket: Third
771,Ticket: Third
126,Ticket: Third


In [149]:
row = 0
match = 1 # second match

ti['ticket'].str.extractall('([a-zA-Z]+: [a-zA-Z]+)').loc[row, match]

0    Port: Southampton
Name: (0, 1), dtype: object

In [150]:
ti['ticket'].str.extract('([a-zA-Z]+tow?n)')

,0
0,Southampton
1,NaN
2,Southampton
3,Southampton
4,Southampton
...,...
886,Southampton
887,Southampton
888,Southampton
889,NaN


* EXTRA: 
    * escaping
        * How do I say, literally, the `.` symbol?
        * `\.`
    

In [160]:
ti['ticket'].str.extract('(\$ [0-9]+\.[0-9]+)').sample(2)

,0
718,$ 15.5
620,$ 14.4542


* positional matching
    * `^` means **at the beginning**
    * `$` means **at the end**

In [158]:
ti['ticket'].str.extractall('([a-zA-Z]+: [a-zA-Z]+;$)').sample(1)

,,0
,match,
160,0,Port: Southampton;


In [159]:
ti['ticket'].str.extractall('(^[a-zA-Z]+: [a-zA-Z]+;)').sample(1)

,,0
,match,
746,0,Ticket: Third;


## Next Steps

* review a "Regex Cheat Sheet"
    * also, eg., https://en.wikipedia.org/wiki/Regular_expression#Examples

## Exercise (30 min)

* find all the words in the tickets 
    * HINT: a word is a repeated letter followed by a space or a colon
    * HINT: `[ :]` means a space or a colon
* find all the USD prices
    * HINT: ``` \$ ``` and repeated numbers 
    
* find all the high-price tickets
    * HINT: consider `\$`, tripple-digit number, `\.`